In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
from tqdm import tqdm

BASEDIR = 'rawdata'
OUTPUTDIR = 'datasets/NEU-DET-YOLO'

CLASSES = ['crazing', 'inclusion', 'patches', 'pitted_surface', 'rolled-in_scale', 'scratches']

def convert_to_yolo_format(xml_file, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    yolo_lines = []
    for obj in root.findall('object'):
        name = obj.find('name').text
        if name not in CLASSES: continue
        class_id = CLASSES.index(name)
        bndbox = obj.find('bndbox')
        xmin = float(bndbox.find('xmin').text)
        ymin = float(bndbox.find('ymin').text)
        xmax = float(bndbox.find('xmax').text)
        ymax = float(bndbox.find('ymax').text)
        xcenter = (xmin + xmax) / 2 / img_width
        ycenter = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        yolo_lines.append(f'{class_id} {xcenter} {ycenter} {width} {height}')
    return yolo_lines

def process_folder(split_name, source_split_name):
    print(f'Processing {split_name} data...')
    img_source = os.path.join(BASEDIR, source_split_name, 'images')
    xml_source = os.path.join(BASEDIR, source_split_name, 'annotations')
    img_dest = os.path.join(OUTPUTDIR, 'images', split_name)
    label_dest = os.path.join(OUTPUTDIR, 'labels', split_name)
    os.makedirs(img_dest, exist_ok=True)
    os.makedirs(label_dest, exist_ok=True)
    xml_files = [f for f in os.listdir(xml_source) if f.endswith('.xml')]
    for xml_file in tqdm(xml_files):
        try:
            yolo_data = convert_to_yolo_format(os.path.join(xml_source, xml_file), 200, 200)
            txt_filename = xml_file.replace('.xml', '.txt')
            with open(os.path.join(label_dest, txt_filename), 'w') as f:
                f.write('\n'.join(yolo_data))
            img_filename_jpg = xml_file.replace('.xml', '.jpg')
            if os.path.exists(os.path.join(img_source, img_filename_jpg)):
                shutil.copy(os.path.join(img_source, img_filename_jpg), os.path.join(img_dest, img_filename_jpg))
        except Exception as e:
            print(f'Error processing {xml_file}: {e}')

process_folder('train', 'train')
process_folder('val', 'validation')
print(f'Data is ready at {OUTPUTDIR}')

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

In [ ]:
import os
yaml_content = """path: ./datasets/NEU-DET-YOLO
train: images/train
val: images/val

names:
  0: crazing
  1: inclusion
  2: patches
  3: pitted_surface
  4: rolled-in_scale
  5: scratches
"""
with open('data.yaml', 'w') as f:
    f.write(yaml_content)
print('data.yaml has been fixed and recreated!')

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
results = model.train(data='data.yaml', epochs=30, imgsz=200)

In [ ]:
from ultralytics import YOLO
model = YOLO('runs/detect/neu_defect_detector/weights/last.pt')
model.train(resume=True)

In [ ]:
import os
basedir = 'runs/detect'
found_model = False
if os.path.exists(basedir):
    print(f'Searching inside {basedir} ...')
    for folder in os.listdir(basedir):
        potential_path = os.path.join(basedir, folder, 'weights', 'best.pt')
        if os.path.exists(potential_path):
            print(f'FOUND MODEL: {potential_path}')
            found_model = True
            print(f'--- Use this path in your code: model_path = {potential_path.replace(os.sep, "/ ")}')
else:
    print(f'Error: The folder {basedir} does not exist. Did you run the training cell?')
if not found_model and os.path.exists(basedir):
    print('No best.pt found. Your training might have crashed or not finished saving.')

In [ ]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import cv2
import os

model_path = 'runs/detect/train3/weights/best.pt'
model = YOLO(model_path)

results = model.predict(source='datasets/NEU-DET-YOLO/images/val', conf=0.25, save=True)

save_dir = results[0].save_dir
files = os.listdir(save_dir)[:3]

plt.figure(figsize=(15, 5))
for i, file in enumerate(files):
    img = cv2.imread(os.path.join(save_dir, file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.subplot(1, 3, i+1)
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Prediction {i+1}')
plt.show()

In [ ]:
metrics = model.val()
print(f'mAP50 Accuracy: {metrics.box.map50:.3f}')
print(f'Precision: {metrics.box.mp:.3f}')
print(f'Recall: {metrics.box.mr:.3f}')

In [ ]:
# Run this cell to write app.py to disk
# Then launch with: streamlit run app.py
# See app.py in the repository for the full Streamlit dashboard code.

In [ ]:
import os
print(os.getcwd())